In [ ]:
# NEW GA functions that use only dictionaries and simplifed GA
import numpy as np
import pandas as pd

# Initial Population function. Generates a population of initial parameters for sequence 
def initial_population(GA_params, sequence_params_GA):
    population_all = {}
    
    population_size = GA_params['population_size']
    
    for bound_key, bound_value in sequence_params_GA.items():
        population = []
        params = list(bound_value.keys())
        
        for i in range(population_size):
            individual = {}
            
            for param in params:
                bound_param = bound_value[param]
                if isinstance(bound_param, tuple) and len(bound_param) == 2:
                    lower, upper = bound_param
                    if isinstance(lower, int) and isinstance(upper, int):
                        value = np.random.randint(lower, upper)
                    else:
                        value = np.random.uniform(lower, upper)
                else:
                    value = bound_param
                
                individual[param] = value
            
            population.append(individual)
        
        population_all[bound_key] = population
    
    return population_all

# Fitness Function. for calculating the efficiency NOTE This will have to be modified.
# Depending on how you save the data you will have to make the function be able to find and take in the correct data 
# doulbe check that echo_data, input_data and cross_data corrispond to the correct things 
def fitness(echo_data_folder, input_data_folder, cross_data_folder):

    echo_data = pd.read_csv(echo_data_folder + '.csv')
    input_data = pd.read_csv(input_data_folder + '.csv')
    cross_data = pd.read_csv(cross_data_folder + '.csv')
    echo = echo_data['C_2 (V)']
    input = input_data['C_2 (V)']
    cross = cross_data['C_2 (V)']

    # For smoothing data
    kernal_size = 10
    kernal = np.transpose(np.ones(kernal_size)/kernal_size)

    cross = np.convolve(cross,kernal,mode='same')
    input = np.convolve(input,kernal,mode='same')
    echo = np.convolve(echo,kernal,mode='same')

    # Finding the area
    x = np.arange(len(echo))
    area_echo = np.trapezoid(echo,x)
    area_cross = np.trapezoid(cross,x)
    area_input = np.trapezoid(input,x)

    # Finding the efficiency
    fitness = (area_echo-area_input)/area_cross #NOTE these might be wrong order
    fitness = max(fitness, 0)

    return fitness

# Selection Function. Randomly selects (with weighted probability based of fitness) a number of individuals to use for breeding the next generation
def parentselect(population, GA_params, fitness):
    num_parents = GA_params['num_parents']
    population_size = GA_params['population_size']
    
    # Calculate normalised fitness probabilities
    fit = fitness / np.sum(fitness)
    
    parents = {}
    
    for bound_key, individuals in population.items():
        individuals_array = np.array(individuals)
        population_indices = np.arange(len(individuals))
        
        # Weighted selection of parents
        selected_indices = np.random.choice(population_indices, size=num_parents, replace=False, p=fit)
        
        # Gather selected individuals
        parents[bound_key] = individuals_array[selected_indices].tolist()
    
    return parents

# Crossover breeding function. Preforms crossover with parents and until next generation is full.
def crossover(parents, GA_params):
    crossover_rate = GA_params['crossover_rate']
    population_size = GA_params['population_size']
    num_parents = GA_params['num_parents']

    # Initialize offspring population dictionary
    offspring_population = {key: [] for key in parents.keys()}
    current_count = 0

    while current_count < population_size:
        # Randomly shuffle the order of parents
        shuffled_indices = np.random.permutation(num_parents)

        for i in range(0, num_parents, 2):
            if current_count >= population_size:
                break

            parent1_indices = shuffled_indices[i]
            if i + 1 < num_parents:
                parent2_indices = shuffled_indices[i + 1]
            else:
                parent2_indices = shuffled_indices[0]  # Wrap around if odd number of parents

            if np.random.rand() < crossover_rate:
                offspring1 = {}
                offspring2 = {}

                for key in parents.keys():
                    parent1 = parents[key][parent1_indices]
                    parent2 = parents[key][parent2_indices]

                    num_variables = len(parent1)
                    crossover_point = np.random.randint(1, num_variables)

                    offspring1[key] = {**{k: parent1[k] for k in list(parent1.keys())[:crossover_point]},
                                       **{k: parent2[k] for k in list(parent2.keys())[crossover_point:]}}
                    offspring2[key] = {**{k: parent2[k] for k in list(parent2.keys())[:crossover_point]},
                                       **{k: parent1[k] for k in list(parent1.keys())[crossover_point:]}}
                
                for key in offspring1.keys():
                    offspring_population[key].append(offspring1[key])
                current_count += 1
                if current_count < population_size:
                    for key in offspring2.keys():
                        offspring_population[key].append(offspring2[key])
                    current_count += 1
            else:
                # No crossover, offspring are copies of parents
                for key in parents.keys():
                    offspring_population[key].append(parents[key][parent1_indices])
                current_count += 1
                if current_count < population_size:
                    for key in parents.keys():
                        offspring_population[key].append(parents[key][parent2_indices])
                    current_count += 1

    # Ensure the offspring_population is trimmed to the population_size
    for key in offspring_population.keys():
        offspring_population[key] = offspring_population[key][:population_size]

    return offspring_population

# Mutation Function. Mutates the new generation
def mutate(population, GA_params, sequence_params_GA):
    mutation_rate = GA_params['mutation_rate']
    mutation_range = GA_params['mutation_range']
    
    for bound_key, individuals in population.items():
        for individual in individuals:
            for param, value in individual.items():
                if np.random.rand() < mutation_rate:
                    bound_param = sequence_params_GA[bound_key][param]
                    
                    if isinstance(bound_param, tuple) and len(bound_param) == 2:
                        lower_bound, upper_bound = bound_param
                        range_span = upper_bound - lower_bound
                        mutation_amount = range_span * mutation_range * (np.random.uniform(-1, 1))
                        
                        if isinstance(value, int):
                            new_value = int(value + mutation_amount)
                        else:
                            new_value = value + mutation_amount
                        
                        # Ensure the new value is within bounds
                        new_value = np.clip(new_value, lower_bound, upper_bound)
                        individual[param] = new_value
                    else:
                        # If the parameter is a constant, do not mutate
                        individual[param] = value
                        
    return population

# Final solution function. Finds the best solution after the code has finished running 
def optimal_solution(fit, params_GA_save, GA_params):
    index_of_max = np.unravel_index(np.argmax(fit, axis=None), fit.shape)
    i, j = index_of_max
    n = i * GA_params['population_size'] + j

    optimal_params = {key: params_GA_save[key][n] for key in params_GA_save}
    return optimal_params